In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config

In [9]:
def print_sent_ie(sp_i, d):
    print("sentence{}".format(sp_i))
    print(d['SENTS'][sp_i])
    for e in d['DIE']['NER']:
        if  d['SENTS'][sp_i]['start'] <= e['char_b'] and e['char_e'] <= d['SENTS'][sp_i]['end']:
            print(e)

In [10]:
data = json_load("sp_pred.json")

In [15]:
d = data[2]
for q in d['QUESTIONS']:
    print(q['QID'])
    print(q['QTEXT_CN'])
    print("atype:{}".format(q['ATYPE']))
    print("SHINT:{}".format(q['SHINT']))
    print("sp:{}".format(q['sp']))
    all_set = set(q['SHINT']) | set(q['sp'])
    for sp_i in range(min(all_set), max(all_set) + 1):
        print_sent_ie(sp_i, d)

D003Q03
在「乌台诗案」中，有哪些人出面救了苏东坡？
atype:Person
SHINT:[19, 20, 21]
sp:[15, 20]
sentence15
{'text': '苏轼参加救灾。', 'start': 197, 'end': 204}
sentence16
{'text': '\n元丰二年（1079年），', 'start': 204, 'end': 217}
{'id': 'D0-S3-M0', 'string': '元丰', 'type': 'MISC', 'char_b': 205, 'char_e': 207}
{'id': 'D0-S3-M1', 'string': '二年', 'type': 'DATE', 'char_b': 207, 'char_e': 209}
{'id': 'D0-S3-M2', 'string': '1079年', 'type': 'DATE', 'char_b': 210, 'char_e': 215}
sentence17
{'text': '四十三岁时，', 'start': 217, 'end': 223}
{'id': 'D0-S3-M3', 'string': '四十三', 'type': 'NUMBER', 'char_b': 217, 'char_e': 220}
{'id': 'D0-S3-M4', 'string': '岁', 'type': 'MISC', 'char_b': 220, 'char_e': 221}
sentence18
{'text': '因乌台诗案入狱，', 'start': 223, 'end': 231}
sentence19
{'text': '几死，因为写文章向朝廷诀别，', 'start': 231, 'end': 245}
{'id': 'D0-S3-M5', 'string': '几', 'type': 'NUMBER', 'char_b': 231, 'char_e': 232}
sentence20
{'text': '太皇太后曹氏、王安礼等人出面力挽，', 'start': 245, 'end': 262}
{'id': 'D0-S3-M6', 'string': '王安礼', 'type': 'PER', 'char_b': 252, 'c